# Sound Switch

# Imports

In [1]:
%reset -f
%matplotlib inline
%config InlineBackend.figure_format = "retina" # High-res graphs (rendered irrelevant by svg option below)
%config InlineBackend.print_figure_kwargs = {"bbox_inches": "tight"} # No extra white space
%config InlineBackend.figure_format = "svg" # 'png' is default
 
import warnings
warnings.filterwarnings("ignore") # Because we are adults
from IPython.core.debugger import set_trace
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
from dfply import *
 
# iPyPublish imports
# from ipypublish.scripts.ipynb_latex_setup import *
# from IPython.display import SVG, display, Markdown

# Experiment 1

## Methods

### Stimuli
- Two types of sound stimuli: tones and guitar chords
    - Beeps alternate between pure C4 tone (261.626 Hz) and pure G tone (391.995 Hz)
    - Guitar chords alternate between a C chord and a G chord strummed on guitar, taken from [here](https://www.apronus.com/music/onlineguitar.htm), which were subsequently recorded and amplified (25.98 db and 25.065 db respectively) using Audacity version 2.3.0 (using Effect --> Amplify) via Windows 10's Stereo Mix drivers.
- 20 sounds per stimulus
- Crossfade of 50 ms between sounds
- Each stimulus was 10,000 milliseconds long, with an additional 150 ms of silence at the start of the recording.
- 9 switch rates between 0.1 and 0.9
- 10 stimuli for each switch rate, for a total of 180 stimuli, all presented as a single block. 30 frame ITI.
- 20 repeated stimuli chosen via the same random choice for each subject (presentation order was via a unique random choice for each subject, however).
- One practice beep stimulus at 0.5 switch rate presented at the start of the experiment.


## Results

### Read in data

In [2]:
import glob, os

separator = r"\t"
data_dir = "../data"
prefix = "Sound Switch"
extension = ".txt"
search = f"{data_dir}/{prefix}*{extension}"

# Get each subject's data and concatenate into 
# one larger DataFrame
all_data = pd.DataFrame()
for file_name in glob.glob(search):
    subject_data = pd.read_csv(file_name, sep=separator)
    if all_data.shape == (0, 0):
        all_data = subject_data
    else: 
        all_data = all_data.append(subject_data)

all_data.reset_index(inplace=True, drop=True)
all_data.head()

Subject ID  Condition  Block #  Trial #  Switch Rate  Exemplar  \
0           1  patterned        1        1          0.3        10   
1           1  patterned        1        2          0.1        17   
2           1  patterned        1        3          0.1        18   
3           1  patterned        1        4          0.7        10   
4           1  patterned        1        5          0.9         9   

                                           File Name  Rating     RT  \
0  ../stimuli/combined/switch-0.3_chunk-500_C_G_a...      41  7.309   
1  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...     100  2.725   
2  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...      71  3.493   
3  ../stimuli/combined/switch-0.7_chunk-500_C_G_a...      68  2.446   
4  ../stimuli/combined/switch-0.9_chunk-500_C_G_a...     100  2.178   

                              Rating History  Trial Duration  Block Duration  \
0     [(None, 0.0), (41, 6.48), (41, 7.309)]       18.222353       18.223059   
1  [(None, 0.0), (100, 1.817), (100, 2.725)]       13.631996       31.859845   
2    [(None, 0.0), (71, 2.784), (71, 3.493)]       14.413484       46.279105   
3    [(None, 0.0), (68, 1.726), (68, 2.446)]       13.391476       59.676363   
4  [(None, 0.0), (100, 1.333), (100, 2.178)]       13.100105       72.781313   

   Session Duration  Experiment Duration        Start Date  \
0       3670.838750            19.814849  2019_Feb_14_1305   
1       3684.475536            33.451635  2019_Feb_14_1305   
2       3698.894796            47.870896  2019_Feb_14_1305   
3       3712.292055            61.268155  2019_Feb_14_1305   
4       3725.397003            74.373104  2019_Feb_14_1305   

                Experiment Testing Location Experimenter Initials  \
0  Sound Switch 12-20-2018       318-laptop                   bpl   
1  Sound Switch 12-20-2018       318-laptop                   bpl   
2  Sound Switch 12-20-2018       318-laptop                   bpl   
3  Sound Switch 12-20-2018       318-laptop                   bpl   
4  Sound Switch 12-20-2018       318-laptop                   bpl   

  Subject Initials  
0              WPH  
1              WPH  
2              WPH  
3              WPH  
4              WPH

## Determine stimulus type

In [3]:
def get_stimulus_type(row):
    """
    All stimuli under 10 are guitar; the rest are tones.
    """
    return "guitar" if row["Exemplar"] < 10 else "tone"

all_data["Stimulus Type"] = all_data.apply(get_stimulus_type, axis=1)
all_data.head()

Subject ID  Condition  Block #  Trial #  Switch Rate  Exemplar  \
0           1  patterned        1        1          0.3        10   
1           1  patterned        1        2          0.1        17   
2           1  patterned        1        3          0.1        18   
3           1  patterned        1        4          0.7        10   
4           1  patterned        1        5          0.9         9   

                                           File Name  Rating     RT  \
0  ../stimuli/combined/switch-0.3_chunk-500_C_G_a...      41  7.309   
1  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...     100  2.725   
2  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...      71  3.493   
3  ../stimuli/combined/switch-0.7_chunk-500_C_G_a...      68  2.446   
4  ../stimuli/combined/switch-0.9_chunk-500_C_G_a...     100  2.178   

                              Rating History  Trial Duration  Block Duration  \
0     [(None, 0.0), (41, 6.48), (41, 7.309)]       18.222353       18.223059   
1  [(None, 0.0), (100, 1.817), (100, 2.725)]       13.631996       31.859845   
2    [(None, 0.0), (71, 2.784), (71, 3.493)]       14.413484       46.279105   
3    [(None, 0.0), (68, 1.726), (68, 2.446)]       13.391476       59.676363   
4  [(None, 0.0), (100, 1.333), (100, 2.178)]       13.100105       72.781313   

   Session Duration  Experiment Duration        Start Date  \
0       3670.838750            19.814849  2019_Feb_14_1305   
1       3684.475536            33.451635  2019_Feb_14_1305   
2       3698.894796            47.870896  2019_Feb_14_1305   
3       3712.292055            61.268155  2019_Feb_14_1305   
4       3725.397003            74.373104  2019_Feb_14_1305   

                Experiment Testing Location Experimenter Initials  \
0  Sound Switch 12-20-2018       318-laptop                   bpl   
1  Sound Switch 12-20-2018       318-laptop                   bpl   
2  Sound Switch 12-20-2018       318-laptop                   bpl   
3  Sound Switch 12-20-2018       318-laptop                   bpl   
4  Sound Switch 12-20-2018       318-laptop                   bpl   

  Subject Initials Stimulus Type  
0              WPH          tone  
1              WPH          tone  
2              WPH          tone  
3              WPH          tone  
4              WPH        guitar

## Determine if trial is a repeated stimulus

In [4]:
num_trials = 200
num_repeated_trials = 20 # at end of task
start_repeated_trial_index = num_trials - num_repeated_trials + 1

all_data["Repeat Trial"] = all_data["Trial #"] >= start_repeated_trial_index
all_data.head()

Subject ID  Condition  Block #  Trial #  Switch Rate  Exemplar  \
0           1  patterned        1        1          0.3        10   
1           1  patterned        1        2          0.1        17   
2           1  patterned        1        3          0.1        18   
3           1  patterned        1        4          0.7        10   
4           1  patterned        1        5          0.9         9   

                                           File Name  Rating     RT  \
0  ../stimuli/combined/switch-0.3_chunk-500_C_G_a...      41  7.309   
1  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...     100  2.725   
2  ../stimuli/combined/switch-0.1_chunk-500_C_G_a...      71  3.493   
3  ../stimuli/combined/switch-0.7_chunk-500_C_G_a...      68  2.446   
4  ../stimuli/combined/switch-0.9_chunk-500_C_G_a...     100  2.178   

                              Rating History  ...  Block Duration  \
0     [(None, 0.0), (41, 6.48), (41, 7.309)]  ...       18.223059   
1  [(None, 0.0), (100, 1.817), (100, 2.725)]  ...       31.859845   
2    [(None, 0.0), (71, 2.784), (71, 3.493)]  ...       46.279105   
3    [(None, 0.0), (68, 1.726), (68, 2.446)]  ...       59.676363   
4  [(None, 0.0), (100, 1.333), (100, 2.178)]  ...       72.781313   

   Session Duration  Experiment Duration        Start Date  \
0       3670.838750            19.814849  2019_Feb_14_1305   
1       3684.475536            33.451635  2019_Feb_14_1305   
2       3698.894796            47.870896  2019_Feb_14_1305   
3       3712.292055            61.268155  2019_Feb_14_1305   
4       3725.397003            74.373104  2019_Feb_14_1305   

                Experiment Testing Location Experimenter Initials  \
0  Sound Switch 12-20-2018       318-laptop                   bpl   
1  Sound Switch 12-20-2018       318-laptop                   bpl   
2  Sound Switch 12-20-2018       318-laptop                   bpl   
3  Sound Switch 12-20-2018       318-laptop                   bpl   
4  Sound Switch 12-20-2018       318-laptop                   bpl   

  Subject Initials Stimulus Type Repeat Trial  
0              WPH          tone        False  
1              WPH          tone        False  
2              WPH          tone        False  
3              WPH          tone        False  
4              WPH        guitar        False  

[5 rows x 21 columns]

## Get subject reliability

In [7]:
subject_reliability_df = pd.DataFrame(
    index=[],
    columns=[
        "subject_id",
        "condition",        
        "correlation",
        "p-value",
        "fisher_z",
    ],
)
subject_ids = all_data["Subject ID"].unique()
for subject_id in subject_ids:
    subject_data = all_data >> mask(X["Subject ID"] == subject_id)
    subject_data.sort_values(by=["File Name"], inplace=True)
    repeated_stimuli = (
        subject_data 
        >> mask(X["Repeat Trial"] == True)
        >> select(["File Name"])
    )
    
    first_ratings_data = subject_data[
        (subject_data["File Name"].isin(repeated_stimuli["File Name"]))
        & (subject_data["Repeat Trial"] == False)
    ]["Rating"].tolist()
    
    second_ratings_data = subject_data[
        (subject_data["File Name"].isin(repeated_stimuli["File Name"]))
        & (subject_data["Repeat Trial"] == True)
    ]["Rating"].tolist()
    
    subject_reliability_corr = scipy.stats.pearsonr(
        first_ratings_data, second_ratings_data
    )
    
    # Needs to be in arrays for pandas
    # XXX break down by stimulus type?
    this_corr_row = {
        "subject_id": [subject_id],
        "condition": [subject_data["Condition"].iloc[0]],        
        "correlation": [subject_reliability_corr[0]],
        "p-value": [subject_reliability_corr[1]],
        "fisher_z": [np.arctanh(subject_reliability_corr[0])],
    }
    this_corr_row = pd.DataFrame.from_dict(this_corr_row)
    subject_reliability_df = subject_reliability_df.append(this_corr_row)
    
subject_reliability_df

subject_id  condition  correlation       p-value  fisher_z
0          1  patterned     0.104783  6.601953e-01  0.105169
0          2  beautiful     0.952372  1.007080e-10  1.856693
0          3  patterned     0.633475  2.712090e-03  0.747199
0          4  beautiful     0.206562  3.822442e-01  0.209577
0          5  patterned     0.795481  2.754297e-05  1.086183
0          6  beautiful    -0.176351  4.570326e-01 -0.178214
0          7  patterned     0.300609  1.978025e-01  0.310189

# Sanity checks

## Same number of trials for all subjects?
All subjects should have exactly 200 trials.

In [68]:
subject_ids = all_data["Subject ID"].unique()
for subject_id in subject_ids:
    subject_data = all_data >> mask(X["Subject ID"] == subject_id)
    print(subject_data.shape)

(200, 21)
(200, 21)
(200, 21)
(200, 21)
(200, 21)
(200, 21)
(200, 21)


## All stimuli same length?
Should all be 10,100 ms long.

In [75]:
from pydub import AudioSegment

stimulus_dir = "../stimuli/combined"
prefix = "switch-"
extension = ".mp3"
search = f"{stimulus_dir}/{prefix}*{extension}"

song_durations = []
for song_name in glob.glob(search):
    song = AudioSegment.from_mp3(song_name)
    song_durations.append(len(song))
    
print(set(song_durations))

{10100}


# Scrap